In [1]:
#import 
from models.DynamicalAE import *
import pickle
import torch
from torch import nn
import numpy as np
from matplotlib import pyplot as plt
import os
#from data_generation.geom_utils import graph_distance
from settings.custom_settings import CustomSettings
from data_generation.generate_trajectory import Trajectory
from copy import deepcopy
from mpl_toolkits.mplot3d import axes3d
from training.train_ae import preprocessing
import pandas as pd
from matplotlib.path import Path
import matplotlib.patches as patches
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
LATENT_SPACE_DIM = 2

GRAPH_MODEL_PATH = "./saved_models/dynAE_graph_nonoise_2D" #5mazes_Graph_3dAE_excluding1
GRAPH_EXP_NAME = "experiment14"  

In [3]:
#Load data and model


#GRAPH
with open(GRAPH_MODEL_PATH + '/test_data', 'rb') as file:
    X_test_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + '/train_data', 'rb') as file:
    X_train_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + '/nonoise_test_data', 'rb') as file:
    X_n_test_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + '/nonoise_train_data', 'rb') as file:
    X_n_train_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + "/test_data_traj", 'rb') as file:
    traj_test_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + "/train_data_traj", 'rb') as file:
    traj_train_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + "/test_maze_config", 'rb') as file:
    test_maze_config_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + "/train_maze_config", 'rb') as file:
    train_maze_config_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + "/train_edge", 'rb') as file:
    train_edge_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + "/test_edge", 'rb') as file:
    test_edge_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + "/v_train", 'rb') as file:
    v_train_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + "/v_test", 'rb') as file:
    v_test_g = pickle.load(file)
with open(GRAPH_MODEL_PATH + "/loss_logs", 'rb') as file:
    loss_logs_g = pickle.load(file)

path = "./data_generation/generated_data/" + GRAPH_EXP_NAME
with open(path + "/maze.pkl", 'rb') as file:
    maze = pickle.load(file) #Maze is common to both
with open(path + "/trajectory.pkl", 'rb') as file:
    traj_g = pickle.load(file)
with open(path + "/placeCells.pkl", 'rb') as file:
    placeCells_g = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: './saved_models/dynAE_graph_nonoise_2D/test_data'

In [ ]:
#load model
with open(GRAPH_MODEL_PATH + "/model", 'rb') as file:
    GM = pickle.load(file)

GM.eval()

LS_test_g = GM.encoder(X_test_g[:, 0, :])
LS_next_test_g = GM.f(LS_test_g, test_edge_g[:, 0, :], v_test_g[:, 0, :])
LS_rec = GM.decoder(LS_next_test_g)

LS_train_g = GM.encoder(X_train_g[:, 0, :])
LS_next_train_g = GM.f(LS_train_g, train_edge_g[:, 0, :], v_train_g[:, 0, :])
LS_rec_train = GM.decoder(LS_next_train_g)
#rec_test_g = graph_model.decoder(latent_space_test_g)
#rec_train_g = graph_model.decoder(latent_space_train_g)

LS_rec = LS_rec.detach().numpy()
LS_rec_train = LS_rec_train.detach().numpy()

In [ ]:
plt.plot(GM.lossLogs)


In [ ]:
idx = np.argsort(LS_rec[:, 0])
L=np.arange(len(idx))
plt.scatter(L, X_test_g.detach().numpy()[idx, 1, 0])
plt.scatter(L, LS_rec[idx, 0])

In [ ]:
idx = np.argsort(LS_rec_train[:, 0])
L=np.arange(len(idx))
plt.scatter(L, X_train_g[idx, 1, 0])
plt.scatter(L, LS_rec_train[idx, 0])

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go


fig = go.Figure()
df = pd.DataFrame(LS_train_g.detach().numpy())
fig.add_scatter(x=df[0], y=df[1],mode='markers', name="X(n)") #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )
df = pd.DataFrame(LS_next_train_g.detach().numpy())
fig.add_scatter(x=df[0], y=df[1],mode='markers', name="X(n+1)") #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )

fig.show()

In [ ]:
#TRaj towards goal 0


#X(n)
idx = np.where(train_edge_g[:, 0, :] == 2)[1]
df = pd.DataFrame(LS_train_g.detach().numpy()[idx, :])
fig = go.Figure()
#df = df.sample(n=100000)
fig.add_scatter(x=df[0], y=df[1],mode='markers', name="X(n)") #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )

LS_prev = LS_train_g[idx, :]
for i in range(5): 
    #X(n+1)
    LS_next = GM.f(LS_prev, train_edge_g[:, 0, idx], v_train_g[:, 0, idx])
    df = pd.DataFrame(LS_next.detach().numpy())
    fig.add_scatter(x=df[0], y=df[1],mode='markers', name="X(n+%s)"%(i+1)) #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )
    LS_prev = LS_next


fig.show()

In [ ]:
#Plot home goal/ graph distance
with open("./dgg_dyn2D", 'rb') as file:
    dgg_dyn2D = pickle.load(file)
    
h = np.array(maze.homes) + 0.5
h_fr = np.empty((maze.nb_of_trials, 100))
for i in range(maze.nb_of_trials):
    h_fr[i, :] = preprocessing(placeCells_g.fire(maze, i, np.array([h[i]]).T), placeCells_g)
h_latent = GM.encoder(torch.Tensor(h_fr)).detach().numpy()
    
df = pd.DataFrame(LS_train_g.detach().numpy())
fig = go.Figure()
marker_dict = dict(size=3,
    color=dgg_dyn2D,  # set color to an array/list of desired values
    colorscale='Reds')
fig.add_scatter(x=df[0], y=df[1],mode='markers', marker=marker_dict,  name="X(n)") #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )
marker_dict = dict(size=10)
fig.add_scatter(x=h_latent[:, 0], y=h_latent[:, 1],  marker_symbol="cross",mode='markers',marker=marker_dict,  name="HOMES") #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )


col_list = ["Blues", "Greens"]

LS_prev = LS_train_g
h_prev = torch.Tensor(h_latent)
for i in range(1): 
    #X(n+1)
    marker_dict = dict(size=3,
    color=dgg_dyn2D,  # set color to an array/list of desired values
    colorscale = col_list[i])
    
    LS_next = GM.f(LS_prev, train_edge_g[:, 0, :], v_train_g[:, 0, :])
    h_next =  GM.f(h_prev, torch.Tensor([np.arange(maze.nb_of_trials)]), torch.Tensor(np.ones((1, maze.nb_of_trials))*0.1)).detach().numpy()
    df = pd.DataFrame(LS_next.detach().numpy())
    fig.add_scatter(x=df[0], y=df[1],mode='markers', marker=marker_dict, name="X(n+%s)"%(i+1)) #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )
    marker_dict = dict(size=10)
    fig.add_scatter(x=h_next[:, 0], y=h_next[:, 1],  marker_symbol="cross",mode='markers',marker=marker_dict,  name="HOMES") #, color = "Euclidean distance from home", title = "Euclidean Hyp: latent space" )

    LS_prev = LS_next   
    h_prev = torch.Tensor(h_next)
    

fig.show()

In [ ]:
maze.nb_of_trials

In [ ]:
#try 2d
#simulate 2d trajectory 

In [ ]:
train_edge_g[:, 0, :]